Implementation of CNN using TensorFlow and OpenCV:

In [1]:
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
import os
from tensorflow.keras import Sequential
from tensorflow.keras import layers

c:\Users\justi\anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
def load_yolo_data(data_dir):
    """
    load_yolo_data: Reads data from "data_dir", scales image and assigns label 
                    (class) based on .txt file (1 is weed, 0 is plant)

    :param data_dir: The path of the data that the function should read from
    :return: Images and the corresponding labels as numpy arrays for preprocessing
    """
    images = []
    labels = []
    
    for filename in os.listdir(data_dir):
        if filename.endswith('.jpeg') or filename.endswith('.jpg'):
            img_path = os.path.join(data_dir, filename)
            image = cv2.imread(img_path)
            image_resized = cv2.resize(image, (224, 224))
            image_rgb = cv2.cvtColor(image_resized, cv2.COLOR_BGR2RGB)
            images.append(image_rgb)
            
            annotation_path = os.path.join(data_dir, filename.replace('.jpeg', '.txt').replace('.jpg', '.txt'))
            with open(annotation_path, 'r') as file:
                yolo_data = file.readline().split()
                class_label = int(yolo_data[0])
                labels.append(class_label)
    
    return np.array(images), np.array(labels)

preprocess_images:

Just preprocesses pixels of images to a scale of [0, 1].

In [4]:
def preprocess_images(images):
    """
    preprocess_images: Preprocesses images by scaling pixel values (to a scale of [0, 1]).

    :param images: Numpy array of images
    :return: Scaled images
    """
    images = images.astype('float32') / 255.0
    return images

In [5]:
data_dir = './data' # Replace with the path where the data is contained
images, labels = load_yolo_data(data_dir)
images = preprocess_images(images)

# Split the data into train and test sets:
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)
# When we combine our data, we can create a validation split too

In [11]:
print(len(X_train))
print(len(X_test))

1040
260


In [12]:
def cnn_model(input_shape=(224, 224, 3)):
    """
    cnn_model: Constructs and returns a Convolutional Neural Network

    :param input_shape: This shape should match the size of the resized data in the
                        preprocessing step
    :return: model which corresponds to constructed CNN
    """
    model = Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dense(2, activation='softmax'))
     
    return model

In [13]:
model = cnn_model()
model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 111, 111, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 54, 54, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 26, 26, 128)      

In [14]:
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'],
             )

In [15]:
history = model.fit(X_train,
                    y_train,
                    epochs=10, 
                    validation_split=.3,
                   )

Epoch 1/10


23/23 [==============================] - 16s 609ms/step - loss: 2.1225 - accuracy: 0.5192 - val_loss: 0.6815 - val_accuracy: 0.5353
Epoch 2/10
23/23 [==============================] - 13s 582ms/step - loss: 0.6136 - accuracy: 0.6538 - val_loss: 0.4568 - val_accuracy: 0.8558
Epoch 3/10
23/23 [==============================] - 13s 588ms/step - loss: 0.4828 - accuracy: 0.7871 - val_loss: 0.4149 - val_accuracy: 0.8301
Epoch 4/10
23/23 [==============================] - 14s 598ms/step - loss: 0.3916 - accuracy: 0.8544 - val_loss: 0.4330 - val_accuracy: 0.8558
Epoch 5/10
23/23 [==============================] - 14s 595ms/step - loss: 0.4163 - accuracy: 0.8544 - val_loss: 0.3993 - val_accuracy: 0.8558
Epoch 6/10
23/23 [==============================] - 15s 645ms/step - loss: 0.3432 - accuracy: 0.8723 - val_loss: 0.2941 - val_accuracy: 0.9071
Epoch 7/10
23/23 [==============================] - 15s 633ms/step - loss: 0.2779 - accuracy: 0.9011 - val_loss: 0.2659 - val_accuracy: 0.91

In [16]:
test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy[1]}")

9/9 [==============================] - 1s 143ms/step - loss: 0.1974 - accuracy: 0.9385
Test Accuracy: 0.9384615421295166
